In [1]:
%load_ext autoreload
%autoreload 2

import torch
torch.__version__

KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch
import random
import pickle
import sys
import os
import glob
import pdb
from tqdm.notebook import tqdm
sys.path.insert(0, '/home/jupyter/VLP/pythia')
sys.path.insert(0, '/home/jupyter/VLP/')

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTrainingLossMask
from pytorch_pretrained_bert.optimization import BertAdam
from pathlib import Path
import pandas as pd
from vlp.loader_utils import batch_list_to_batch_tensors
import vlp.seq2seq_loader as seq2seq_loader
import PIL
from vlp.lang_utils import language_eval

from fastai.vision.all import *

from vlp_processor import PreprocessVLP
import pythia.tasks.processors as pythia_proc

from util import *

In [ ]:
class ArgDummy(dict):
    def __getattr__(self, attr):
        return self[attr]
args = ArgDummy()
DATA_ROOT = '/mnt/ssd/data'
HATE_FEAT_PATH = Path('/home/jupyter/hateful_features_p2/region_feat_gvd_wo_bgd')

args['bert_model'] = 'bert-base-cased' #Bert pre-trained model selected
args['seed'] = 123 #random seed for initialization
args['len_vis_input'] = 100
args['max_tgt_length'] = 100#20 #maximum length of target sequence
args['region_det_file_prefix'] = 'feat_cls_1000/coco_detection_vg_100dets_gvd_checkpoint_trainval'
args['output_dir'] ='tmp'
args['drop_prob'] = 0.1
args['model_recover_path'] = './checkpoints/vqa2_g2_lr2e-5_batch512_ft_from_s0.75_b0.25/model.19.bin'
args['image_root'] = f'{DATA_ROOT}/flickr30k/region_feat_gvd_wo_bgd/'
args['region_bbox_file'] =f'{DATA_ROOT}/flickr30k/region_feat_gvd_wo_bgd/flickr30k_detection_vg_thresh0.2_feat_gvd_checkpoint_trainvaltest.h5'
args['do_lower_case'] = True
args.region_bbox_file = os.path.join(args.image_root, args.region_bbox_file)
args.region_det_file_prefix = os.path.join(args.image_root, args.region_det_file_prefix)
args.max_seq_length = args.max_tgt_length + args.len_vis_input + 3 # +3 for 2x[SEP] and [CLS]

In [ ]:
device = torch.device('cuda')
# fix random seed
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)
tokenizer.max_len = args.max_seq_length

In [ ]:
class HateStem(torch.nn.Module):
    
    def __init__(self, vlp):
        super(HateStem, self).__init__()
        self.vis_embed = vlp.vis_embed #Linear->ReLU->Linear->ReLU->dropout
        self.vis_pe_embed = vlp.vis_pe_embed #Linear->ReLU->dropout
        self.bert = vlp.bert # pytorch_pretrained_bert.modeling.BertModel
        self.len_vis_input = vlp.len_vis_input
        
    
    def forward(self, vis_feats, vis_pe, input_ids, token_type_ids=None, attention_mask=None):
        vis_feats = self.vis_embed(vis_feats) # image region features
        vis_pe = self.vis_pe_embed(vis_pe) # image region positional encodings

        sequence_output, pooled_output = self.bert(vis_feats, vis_pe, input_ids, token_type_ids,
            attention_mask, output_all_encoded_layers=False, len_vis_input=self.len_vis_input)
        #print(sequence_output.shape, pooled_output.shape)
        vqa2_embed = sequence_output[:, 0]*sequence_output[:, self.len_vis_input+1]
        return vqa2_embed
        #return sequence_output


In [ ]:
def create_head(nf, n_out, lin_ftrs=None, ps=0.5, bn_final=False, lin_first=False, ):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and out `n_out` classes."
    lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [nf] + lin_ftrs + [n_out]
    ps = L(ps)
    if len(ps) == 1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    layers = []
    layers = [Flatten()]
    if lin_first: layers.append(nn.Dropout(ps.pop(0)))
    for ni,no,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], ps, actns):
        layers += LinBnDrop(ni, no, bn=True, p=p, act=actn, lin_first=lin_first)
    if lin_first: layers.append(nn.Linear(lin_ftrs[-2], n_out))
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    return nn.Sequential(*layers)

In [ ]:
class HateClassifier(torch.nn.Module):
     def __init__(self, stem):
        super(HateClassifier, self).__init__()
        self.stem = stem
        self.classifier = create_head(768,2, ps=.5)
     #def forward(self, id, vis_feats, vis_pe, input_ids, token_type_ids, attention_mask):  
     def forward(self, params):  
        id, vis_feats, vis_pe, input_ids, token_type_ids, attention_mask = params
        embs = self.stem(vis_feats, vis_pe, input_ids, token_type_ids, attention_mask)
        return self.classifier(embs)

In [ ]:
def new_model():
    hate_stem = torch.load('checkpoints/lm_stem20drop.pth')
    
    return  HateClassifier(hate_stem).cuda()


In [ ]:
PHASE_2 = Path('/home/jupyter/hate_phase2')
train = pd.read_json(PHASE_2/'train.jsonl', lines=True)
dev_seen = pd.read_json(PHASE_2/'dev_seen.jsonl', lines=True)
dev_unseen = pd.read_json(PHASE_2/'dev_unseen.jsonl', lines=True)
test_seen = pd.read_json(PHASE_2/'test_seen.jsonl', lines=True)
test_unseen = pd.read_json(PHASE_2/'test_unseen.jsonl', lines=True)

In [ ]:
# data = pd.concat([train, dev_seen, dev_unseen, test_seen, test_unseen])
# data['label']=1
# data.dtypes

# df_train = pd.DataFrame(data[:10500])
# df_valid = pd.DataFrame(data[10500:])

# df_train['is_valid'] = False
# df_valid['is_valid'] = True

# valid_ids = df_valid.index.tolist()
# valid_wrong = np.random.choice(valid_ids,len(valid_ids)//2)
# valid_wrong[:5]

# shuffled = np.random.permutation(valid_wrong)
# shuffled[:5]

# shuf_texts = df_valid.loc[shuffled]['text'].tolist()
# shuf_texts[:5]

# df_valid.loc[valid_wrong, 'text']= shuf_texts
# df_valid.loc[valid_wrong, 'label'] =0

# data = pd.concat([df_train, df_valid])
# data.to_csv('data_sim.csv')

In [ ]:
data = pd.read_csv('data_sim.csv')
df_train = pd.DataFrame(data[:10500])
df_valid = pd.DataFrame(data[10500:])

In [ ]:
def load(idx, proc, q=None):
    return load_from_row(data.iloc[idx],proc, tokenizer, q)

In [ ]:
region_pref = HATE_FEAT_PATH / 'feat_cls_1000/hateful_vlp_checkpoint_trainval'
bbox_pref = HATE_FEAT_PATH / 'raw_bbox/hateful_vlp_checkpoint_trainval'
id_digits=2

truncate_config={
    'max_len_b': args.max_tgt_length, 'trunc_seg': 'b', 'always_truncate_tail': True}

max_masked = 10
mask_prob = .20
mask_img=True
vis_mask_prob = .20
train_proc = PreprocessVLP(max_masked, mask_prob,
    list(tokenizer.vocab.keys()), tokenizer.convert_tokens_to_ids, args.max_seq_length,
    truncate_config=truncate_config,mask_image_regions=mask_img, vis_mask_prob=vis_mask_prob,
    mode="bi", len_vis_input=args.len_vis_input, 
    region_bbox_prefix=str(bbox_pref), region_det_file_prefix=str(region_pref), id_digits=id_digits,
    load_vqa_ann=True)

val_proc = PreprocessVLP(0, 0,
    list(tokenizer.vocab.keys()), tokenizer.convert_tokens_to_ids, args.max_seq_length,
    truncate_config=truncate_config,mask_image_regions=False, vis_mask_prob=0,
    mode="bi", len_vis_input=args.len_vis_input, 
    region_bbox_prefix=str(bbox_pref), region_det_file_prefix=str(region_pref), id_digits=id_digits,
    load_vqa_ann=True)

In [ ]:
@typedispatch
def show_batch(x:VLPInput, y, samples, ctxs=None, max_n=10, nrows=None, ncols=None, figsize=None, **kwargs):
    if ctxs is None: ctxs = get_grid(min(len(samples), max_n), nrows=nrows, ncols=ncols, figsize=figsize)
    ctxs = show_batch[object](x, y, samples, ctxs=ctxs, max_n=max_n, **kwargs)
    return ctxs

@typedispatch
def show_results(x:VLPInput, y:TensorCategory, samples, outs, ctxs=None, max_n=10, nrows=None, ncols=None, figsize=None, **kwargs):
    if ctxs is None: ctxs = get_grid(min(len(samples), max_n), nrows=nrows, ncols=ncols, add_vert=1, figsize=figsize)
    for i in range(2):
        ctxs = [b.show(ctx=c, **kwargs) for b,c,_ in zip(samples.itemgot(i),ctxs,range(max_n))]
    ctxs = [r.show(ctx=c, color='green' if b==r else 'red', **kwargs)
            for b,r,c,_ in zip(samples.itemgot(1),outs.itemgot(0),ctxs,range(max_n))]
    return ctxs


@typedispatch
def plot_top_losses(x: VLPInput, y:TensorCategory, samples, outs, raws, losses, nrows=None, ncols=None, figsize=None, **kwargs):
    axs = get_grid(len(samples), nrows=nrows, ncols=ncols, add_vert=1, figsize=figsize, title='Prediction/Actual/Loss/Probability')
    for ax,s,o,r,l in zip(axs, samples, outs, raws, losses):
        s[0].show(ctx=ax, **kwargs)
        ax.set_title(f'{o}/{s[1]} / {l.item():.2f} / {r.max().item():.2f}')

In [ ]:
class LoadRow(Transform):
    
    def __init__(self,processor, tokenizer, random_text=False, data = None):
        self.proc = processor
        self.tokenizer = tokenizer
        self.random_text = random_text
        self.data = data
    def encodes(self, x):
        if self.random_text and random.choice((True, False)):
            x = pd.Series(x)
            altloc = random.randint(0, len(self.data)-1)
            alttext = self.data.iloc[altloc].text
            x['text'] = alttext
            x['label'] = 0
        return load_from_row(x, self.proc, self.tokenizer), x.label

In [ ]:
train_load = LoadRow(train_proc, tokenizer, True, df_train)
valid_load = LoadRow(val_proc, tokenizer)

In [ ]:
train_tl = TfmdLists(df_train, train_load)
valid_tl = TfmdLists(df_valid, valid_load)

dls = DataLoaders.from_dsets(train_tl, valid_tl,bs=40, device=device)

In [ ]:


@patch_to(VLPInput)
def show(self, ctx, **kwargs):
    id = self[0].item()
    tit = id_to_text(id, data)
    ctx.text(0,0,tit,ha='left', wrap=True)
    ctx = show_image(PILImage.create(id_to_img_path(id)), ctx=ctx)
    return ctx


In [ ]:
def vlp_splitter(model):
    return L(params(model.stem.vis_embed) + params(model.stem.vis_pe_embed), 
            params(model.stem.bert),
            params(model.classifier))

In [ ]:
model = new_model()
learn = Learner(dls, model,metrics=[accuracy, RocAucBinary()], splitter=vlp_splitter, loss_func = nn.CrossEntropyLoss())

In [ ]:
learn.fit_one_cycle(10, lr_max = 5e-6)

In [ ]:
learn.load('sim_p2')

In [ ]:
import PIL
def show_by_idxs(img_idxs):
    n,m = len(img_idxs)//2, 2
    
    _,axs = plt.subplots(n,m, figsize=(10*m,10*n))
    for ax, idx in zip(axs.flatten(), img_idxs.view(-1)):
        row = data.iloc[idx.item()]
        img_path = PHASE_2 / row['img']
        ax.imshow(PIL.Image.open(img_path))
        ax.axis('off')
        clr = 'red' if row['label']==1 else 'green'
        txt = f'{row["id"]}: {row["text"][:20]}'
        ax.set_title(txt, color=clr) 

In [ ]:
show_by_idxs(torch.tensor(range(6)))

In [ ]:
model = learn.model.eval()

In [ ]:
# first_row = data.iloc[0]

# from tqdm.notebook import tqdm
# bs=64
# all_probs = []
# for batch_num in tqdm(range(len(data)//bs)):
#     batch = []
  
#     for i in range(bs):
#         txt = data.text.iloc[batch_num * bs + i]
#         row_data = load_from_row(first_row, val_proc, tokenizer, txt)
#         batch.append(row_data)
#     b = fa_collate(batch)
#     b = tuple([x.cuda() for x in b])
#     with torch.no_grad():
#         out = model(b)
#     probs = F.softmax(out, dim=1)[:,1]
#     all_probs.append(probs)

# probs_tensor = torch.cat(all_probs)
# probs_tensor.shape

# probs, idxs = probs_tensor.topk(30)
# probs

# data.text.iloc[idxs.cpu().tolist()]

In [ ]:
class LoadImgRow(Transform):
    
    def __init__(self,processor, tokenizer):
        self.proc = processor
        self.tokenizer = tokenizer
    def encodes(self, x):
        return load_from_row(x, self.proc, self.tokenizer, '<SEP>')[1]

In [ ]:
data_tl = TfmdLists(data, LoadImgRow(val_proc, tokenizer))


dls = DataLoaders.from_dsets(data_tl, valid_tl,bs=100, device=device, shuffle=False)

In [ ]:
def img_embs(data_batch):
    feats =[]
    for _, row in data_batch.iterrows():
        bx = load_from_row(row, val_proc, tokenizer, "<SEP>")
        vis_feats = bx[1]
        feats.append(vis_feats.unsqueeze(0))
    feats = torch.cat(feats).cuda()
    print(feats.shape)
    with torch.no_grad():
        vis_embs = model.stem.vis_embed(feats)
    return vis_embs

In [ ]:
all_embs = []
for b in tqdm(dls.train):
    #with torch.no_grad():
    #    vis_embs = model.stem.vis_embed(b)
    vis_embs = b
    all_embs.append(vis_embs.cpu())

In [ ]:
all_embs[0].shape

In [ ]:
embs_v = torch.cat(all_embs,dim=0)
embs_v = embs_v.view(embs_v.shape[0], -1)
embs_v.shape

In [ ]:
def distances_cos(x, b):
    return 1 -F.cosine_similarity(x.unsqueeze(0),b, dim=1)

In [ ]:
dists = 1 -distances_cos(embs_v[8507], embs_v[:8000])

In [ ]:
dists.topk(6)

In [ ]:
show_by_idxs(dists.topk(6)[1])